In [80]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [81]:
%run ./seg_fraud.ipynb

DONE, SET LENGTH: 3234


In [82]:
%run ./seg_normal.ipynb

DONE, SET LENGTH: 13265


In [83]:
%run ./fish_cleaner.ipynb

In [84]:
f1 = open('fraud.txt', 'r')
f2 = open('enron.txt', 'r')

corpus_one = [item for item in f1]
corpus_two = [item for item in f2]

fs = split_set(corpus_one)
ns = split_enron_set(corpus_two)

fraud_set = []
norm_set = []

for i in range(len(fs)):
    fraud_set.append(super_clean(fs[i]))
for i in range(len(ns)):
    norm_set.append(super_clean(ns[i]))    

In [85]:
def bert_screen(x):        #remove emails from datasets that are less than 512 tokens long. 
    z = []
    for i in range(len(x)):
        temp = x[i].split()
        if len(temp) >= 512:
            z.append(x[i])
    return z

In [86]:
print('Length of fraud set before 512 token screening: {}'.format(len(fraud_set)))
print('Length of normal set before 512 token screening: {}'.format(len(norm_set)))

fraud_set = bert_screen(fraud_set)
norm_set = bert_screen(norm_set)

print('\n')
print('Length of fraud set before 512 token screening: {}'.format(len(fraud_set)))
print('Length of normal set before 512 token screening: {}'.format(len(norm_set)))

Length of fraud set before 512 token screening: 3234
Length of normal set before 512 token screening: 13265


Length of fraud set before 512 token screening: 1608
Length of normal set before 512 token screening: 1017


In [87]:
fraud_set = fraud_set[:1000]
norm_set = norm_set[:1000]

In [88]:
def label(fraud, norm):
    new_set = []
    for item in fraud:
        t = (item, 'fraud')
        new_set.append(t)
    for item in norm:
        t = (item, 'norm')
        new_set.append(t)
    return new_set    

In [89]:
x = label(fraud_set,norm_set)

In [90]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
import torch.utils.data

In [91]:
g = pd.DataFrame(x)

In [92]:
pd.set_option('display.max_columns', None)
df = g
print(df.head())


                                                   0      1
0                  FROM:MR. JAMES NGOLA.    URGEN...  fraud
1               Dear Friend,  I am Mr. Ben Sulema...  fraud
2   FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF E...  fraud
3   FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF E...  fraud
4   Dear sir,    It is with a heart full of hope ...  fraud


In [93]:
from collections import Counter
print(Counter(df[1].values))

Counter({'fraud': 1000, 'norm': 1000})


In [94]:
df.dropna(inplace = True)


In [95]:
df = df.rename(columns={0: "text", 1: "type"})

In [96]:
print(len(df))
df.head()

2000


,text,type
0,FROM:MR. JAMES NGOLA. URGEN...,fraud
1,"Dear Friend, I am Mr. Ben Sulema...",fraud
2,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF E...,fraud
3,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF E...,fraud
4,"Dear sir, It is with a heart full of hope ...",fraud


In [97]:
df_fraud = df[df['type'] == 'fraud'] 
df_norm = df[df['type'] == 'norm'] 
df_norm = df_norm.sample(n=len(df_fraud))
df = df_norm.append(df_fraud)
df = df.sample(frac=1, random_state = 24).reset_index(drop=True)

In [98]:
train_data = df.head(1500)
test_data = df.tail(500)

In [99]:
x1 = np.array(train_data['text'])
x2 = np.array(train_data['type'])

y1 = np.array(test_data['text'])
y2 = np.array(test_data['type'])

In [100]:
t1 = []
t2 = []

for i in range(len(x1)):
    t1.append({'text': x1[i], 'type': x2[i]})
    
for i in range(len(y1)):
    t2.append({'text': y1[i], 'type': y2[i]})

In [101]:
train_data = t1
test_data = t2

In [102]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['type']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['type']), test_data)))

In [103]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

Cell above tokenizes text sequences of up to 512 tokens, as that is the max input size of BERT.
Datapoints posessing less than 512 tokens are 



In [104]:
train_y = []
test_y = []
t1 = []
t2 = []

for item in train_labels:
    if str(item) == 'fraud': t1.append(1)
    else: t1.append(0)    

        
for item in test_labels:
    if str(item) == 'fraud': t2.append(1)
    else: t2.append(0)        
    
train_y = np.array(t1)
test_y = np.array(t2)

In [105]:
print("TRAINING DAtA LABEL VECTOR")
print(train_y)
print(test_y)

TRAINING DAtA LABEL VECTOR
[1 1 1 ... 1 1 1]
[1 1 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0
 1 0 1 1 0 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 1 1 1
 0 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 1 0
 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1
 1 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1 1
 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0
 0 1 1 0 0 1 0 1 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0
 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0
 0 1 0 1 1 0 1 0 1 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0
 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 1 0 1 1 0
 1 1 1 0 1 0 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 

In [106]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__() #DEFAULT CONSTRUCTOR INIT
    
        self.bert = BertModel.from_pretrained('bert-base-uncased') #USE BERT BASE FOR LIGHTER LOAD
        self.dropout = nn.Dropout(dropout) #DROPOUT = DEFAULT
        self.linear = nn.Linear(768, 1)    #LINEAR ACTIVATION LAYER, HIDDEN VECTOR LENGTH 768
        self.sigmoid = nn.Sigmoid()        #SIGMOID ACTIVATION LAYER, S SHAPE DECISION BOUNDARY
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba
    #FUNCTION ABOVE DEFINES FLOW OF DATA FOR BERT MODEL.
    #POOLED OUTPUT = DEFAULT BERT POOLED OUTPUT, ABSTRACTION OF DATAPOINT
    #DROPOUT = DEFAULT BERT DROPOUT
    #LINEAR_OUTPUT = LINEAR ACTIVATION WITH HIDDEN SIZE 768
    #FINAL STEP, PUT RESULT INTO SIGMOID ACTIVATION FOR RESULT

In [107]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

#CREATE TENSORS FOR TRAINING. 

In [108]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [109]:
train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=1)
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=1)

STATEMENT BELOW ITERATES THROUGH EACH ITEM IN THE TRAINING DATASET AT ONE EPOCH

In [110]:
BATCH_SIZE = 1
EPOCHS = 1
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

Epoch:  1
0/1500.0 loss: 0.9540891051292419 
Epoch:  1
1/1500.0 loss: 0.8552638292312622 
Epoch:  1
2/1500.0 loss: 0.7748486200968424 
Epoch:  1
3/1500.0 loss: 0.7470849603414536 
Epoch:  1
4/1500.0 loss: 0.7315035462379456 
Epoch:  1
5/1500.0 loss: 0.7136586904525757 
Epoch:  1
6/1500.0 loss: 0.7053364174706596 
Epoch:  1
7/1500.0 loss: 0.7382429763674736 
Epoch:  1
8/1500.0 loss: 0.7337209979693095 
Epoch:  1
9/1500.0 loss: 0.7417612433433532 
Epoch:  1
10/1500.0 loss: 0.7208727273074064 
Epoch:  1
11/1500.0 loss: 0.7124467889467875 
Epoch:  1
12/1500.0 loss: 0.716227439733652 
Epoch:  1
13/1500.0 loss: 0.709198317357472 
Epoch:  1
14/1500.0 loss: 0.6997496088345846 
Epoch:  1
15/1500.0 loss: 0.6916990131139755 
Epoch:  1
16/1500.0 loss: 0.6848906594164231 
Epoch:  1
17/1500.0 loss: 0.6864752802583907 
Epoch:  1
18/1500.0 loss: 0.6987401560733193 
Epoch:  1
19/1500.0 loss: 0.7082746654748917 
Epoch:  1
20/1500.0 loss: 0.6988675367264521 
Epoch:  1
21/1500.0 loss: 0.6919686631722883 


Epoch:  1
177/1500.0 loss: 0.4774705275390925 
Epoch:  1
178/1500.0 loss: 0.4759362056934634 
Epoch:  1
179/1500.0 loss: 0.4747659570640988 
Epoch:  1
180/1500.0 loss: 0.473611689074922 
Epoch:  1
181/1500.0 loss: 0.47248832963325166 
Epoch:  1
182/1500.0 loss: 0.4712655394455123 
Epoch:  1
183/1500.0 loss: 0.4701238465697869 
Epoch:  1
184/1500.0 loss: 0.46895878830471555 
Epoch:  1
185/1500.0 loss: 0.46912934318665533 
Epoch:  1
186/1500.0 loss: 0.4682016712140272 
Epoch:  1
187/1500.0 loss: 0.4668822331314391 
Epoch:  1
188/1500.0 loss: 0.4653381373358782 
Epoch:  1
189/1500.0 loss: 0.46389280612531464 
Epoch:  1
190/1500.0 loss: 0.46402242818740025 
Epoch:  1
191/1500.0 loss: 0.4625569907948375 
Epoch:  1
192/1500.0 loss: 0.46124245681910936 
Epoch:  1
193/1500.0 loss: 0.4604291263007626 
Epoch:  1
194/1500.0 loss: 0.45906574588555554 
Epoch:  1
195/1500.0 loss: 0.45766037702560425 
Epoch:  1
196/1500.0 loss: 0.4566093746780744 
Epoch:  1
197/1500.0 loss: 0.4552609755685835 
Epoch:

Epoch:  1
351/1500.0 loss: 0.3510370060208846 
Epoch:  1
352/1500.0 loss: 0.35585279864055913 
Epoch:  1
353/1500.0 loss: 0.35562756047242106 
Epoch:  1
354/1500.0 loss: 0.3549819291477472 
Epoch:  1
355/1500.0 loss: 0.3542881270640352 
Epoch:  1
356/1500.0 loss: 0.35384802607929006 
Epoch:  1
357/1500.0 loss: 0.35326113414498017 
Epoch:  1
358/1500.0 loss: 0.3525186390680855 
Epoch:  1
359/1500.0 loss: 0.351958088328441 
Epoch:  1
360/1500.0 loss: 0.3512811766436886 
Epoch:  1
361/1500.0 loss: 0.3531514297532772 
Epoch:  1
362/1500.0 loss: 0.3524560807862886 
Epoch:  1
363/1500.0 loss: 0.35195703143356266 
Epoch:  1
364/1500.0 loss: 0.3513706901097951 
Epoch:  1
365/1500.0 loss: 0.3506962750005266 
Epoch:  1
366/1500.0 loss: 0.3501877541316303 
Epoch:  1
367/1500.0 loss: 0.3494911390437704 
Epoch:  1
368/1500.0 loss: 0.34887309382520715 
Epoch:  1
369/1500.0 loss: 0.3482145209972923 
Epoch:  1
370/1500.0 loss: 0.3476005029887202 
Epoch:  1
371/1500.0 loss: 0.3469040060556063 
Epoch:  

Epoch:  1
524/1500.0 loss: 0.28378107463320096 
Epoch:  1
525/1500.0 loss: 0.28337908977423104 
Epoch:  1
526/1500.0 loss: 0.28306696878523485 
Epoch:  1
527/1500.0 loss: 0.28264464030860725 
Epoch:  1
528/1500.0 loss: 0.2823371685027287 
Epoch:  1
529/1500.0 loss: 0.2819658887203572 
Epoch:  1
530/1500.0 loss: 0.2815510601492813 
Epoch:  1
531/1500.0 loss: 0.2811479856491201 
Epoch:  1
532/1500.0 loss: 0.2808051092889139 
Epoch:  1
533/1500.0 loss: 0.2804129950576619 
Epoch:  1
534/1500.0 loss: 0.28010563502941177 
Epoch:  1
535/1500.0 loss: 0.279779274357177 
Epoch:  1
536/1500.0 loss: 0.2793775606324664 
Epoch:  1
537/1500.0 loss: 0.2790592326038278 
Epoch:  1
538/1500.0 loss: 0.2786869277248024 
Epoch:  1
539/1500.0 loss: 0.2783578956499696 
Epoch:  1
540/1500.0 loss: 0.2779663827704083 
Epoch:  1
541/1500.0 loss: 0.27759804291396345 
Epoch:  1
542/1500.0 loss: 0.2773450864187371 
Epoch:  1
543/1500.0 loss: 0.27695604618501796 
Epoch:  1
544/1500.0 loss: 0.2765727607960548 
Epoch: 

Epoch:  1
697/1500.0 loss: 0.23211127254354066 
Epoch:  1
698/1500.0 loss: 0.23184200616097758 
Epoch:  1
699/1500.0 loss: 0.23158919370600156 
Epoch:  1
700/1500.0 loss: 0.23318111762595756 
Epoch:  1
701/1500.0 loss: 0.23291869897214945 
Epoch:  1
702/1500.0 loss: 0.23278035828115082 
Epoch:  1
703/1500.0 loss: 0.23249826664422554 
Epoch:  1
704/1500.0 loss: 0.23227622086803118 
Epoch:  1
705/1500.0 loss: 0.23201594437886736 
Epoch:  1
706/1500.0 loss: 0.23173125583251683 
Epoch:  1
707/1500.0 loss: 0.2314745106544914 
Epoch:  1
708/1500.0 loss: 0.23121324928979192 
Epoch:  1
709/1500.0 loss: 0.23095628345568836 
Epoch:  1
710/1500.0 loss: 0.23071006160234683 
Epoch:  1
711/1500.0 loss: 0.23042808516798646 
Epoch:  1
712/1500.0 loss: 0.2301908948838836 
Epoch:  1
713/1500.0 loss: 0.22992425018801493 
Epoch:  1
714/1500.0 loss: 0.22964215928857976 
Epoch:  1
715/1500.0 loss: 0.22937977728147746 
Epoch:  1
716/1500.0 loss: 0.22911601237214926 
Epoch:  1
717/1500.0 loss: 0.2288562684037

Epoch:  1
869/1500.0 loss: 0.20660863236320773 
Epoch:  1
870/1500.0 loss: 0.20641020265644167 
Epoch:  1
871/1500.0 loss: 0.2062024156981652 
Epoch:  1
872/1500.0 loss: 0.2060674505493895 
Epoch:  1
873/1500.0 loss: 0.2058619129571646 
Epoch:  1
874/1500.0 loss: 0.20565040209037916 
Epoch:  1
875/1500.0 loss: 0.20543951196510973 
Epoch:  1
876/1500.0 loss: 0.205258100560124 
Epoch:  1
877/1500.0 loss: 0.2050685780844174 
Epoch:  1
878/1500.0 loss: 0.20488053651164212 
Epoch:  1
879/1500.0 loss: 0.2046866959532384 
Epoch:  1
880/1500.0 loss: 0.20448448773046424 
Epoch:  1
881/1500.0 loss: 0.20428439351368924 
Epoch:  1
882/1500.0 loss: 0.20408890625864842 
Epoch:  1
883/1500.0 loss: 0.20388858469110405 
Epoch:  1
884/1500.0 loss: 0.2036960655349796 
Epoch:  1
885/1500.0 loss: 0.20351415771663053 
Epoch:  1
886/1500.0 loss: 0.20334857959707667 
Epoch:  1
887/1500.0 loss: 0.20314820579186016 
Epoch:  1
888/1500.0 loss: 0.20297167053984644 
Epoch:  1
889/1500.0 loss: 0.20277216810147078 


Epoch:  1
1041/1500.0 loss: 0.17769641385390467 
Epoch:  1
1042/1500.0 loss: 0.17754547736381582 
Epoch:  1
1043/1500.0 loss: 0.17747026890109496 
Epoch:  1
1044/1500.0 loss: 0.1773212382168861 
Epoch:  1
1045/1500.0 loss: 0.17718056505314478 
Epoch:  1
1046/1500.0 loss: 0.17703680050627835 
Epoch:  1
1047/1500.0 loss: 0.17688985344178917 
Epoch:  1
1048/1500.0 loss: 0.1767371834510559 
Epoch:  1
1049/1500.0 loss: 0.17658969770939578 
Epoch:  1
1050/1500.0 loss: 0.1764395706103911 
Epoch:  1
1051/1500.0 loss: 0.1762956491823772 
Epoch:  1
1052/1500.0 loss: 0.17615015932835518 
Epoch:  1
1053/1500.0 loss: 0.1760085651675766 
Epoch:  1
1054/1500.0 loss: 0.17585995185290468 
Epoch:  1
1055/1500.0 loss: 0.17571513624121013 
Epoch:  1
1056/1500.0 loss: 0.1755655404962705 
Epoch:  1
1057/1500.0 loss: 0.1754246187908663 
Epoch:  1
1058/1500.0 loss: 0.17528605466573108 
Epoch:  1
1059/1500.0 loss: 0.1751527869880621 
Epoch:  1
1060/1500.0 loss: 0.1750103353060129 
Epoch:  1
1061/1500.0 loss: 0

Epoch:  1
1210/1500.0 loss: 0.16133717985932955 
Epoch:  1
1211/1500.0 loss: 0.16121826321885555 
Epoch:  1
1212/1500.0 loss: 0.16110061345620408 
Epoch:  1
1213/1500.0 loss: 0.1609814321565483 
Epoch:  1
1214/1500.0 loss: 0.16086879321469813 
Epoch:  1
1215/1500.0 loss: 0.16074992025249613 
Epoch:  1
1216/1500.0 loss: 0.16062968125644897 
Epoch:  1
1217/1500.0 loss: 0.16052578379169155 
Epoch:  1
1218/1500.0 loss: 0.16041737633136158 
Epoch:  1
1219/1500.0 loss: 0.1603174148699971 
Epoch:  1
1220/1500.0 loss: 0.16019822046142393 
Epoch:  1
1221/1500.0 loss: 0.1600813757972685 
Epoch:  1
1222/1500.0 loss: 0.15997339822565154 
Epoch:  1
1223/1500.0 loss: 0.15985573567237 
Epoch:  1
1224/1500.0 loss: 0.159738944504334 
Epoch:  1
1225/1500.0 loss: 0.15962852245321282 
Epoch:  1
1226/1500.0 loss: 0.15951594662531518 
Epoch:  1
1227/1500.0 loss: 0.15940222158839967 
Epoch:  1
1228/1500.0 loss: 0.15928920064094204 
Epoch:  1
1229/1500.0 loss: 0.15917878237560512 
Epoch:  1
1230/1500.0 loss: 

Epoch:  1
1379/1500.0 loss: 0.14708865831633086 
Epoch:  1
1380/1500.0 loss: 0.14699485701435314 
Epoch:  1
1381/1500.0 loss: 0.14690558006582638 
Epoch:  1
1382/1500.0 loss: 0.14680947514489262 
Epoch:  1
1383/1500.0 loss: 0.14672135787671486 
Epoch:  1
1384/1500.0 loss: 0.1466289599584113 
Epoch:  1
1385/1500.0 loss: 0.14653311638244862 
Epoch:  1
1386/1500.0 loss: 0.14643574457900316 
Epoch:  1
1387/1500.0 loss: 0.14634189439966996 
Epoch:  1
1388/1500.0 loss: 0.14624920971570918 
Epoch:  1
1389/1500.0 loss: 0.14615902780159878 
Epoch:  1
1390/1500.0 loss: 0.146063942696543 
Epoch:  1
1391/1500.0 loss: 0.1459699334013383 
Epoch:  1
1392/1500.0 loss: 0.14587515804913945 
Epoch:  1
1393/1500.0 loss: 0.14578017856193495 
Epoch:  1
1394/1500.0 loss: 0.14568951418055856 
Epoch:  1
1395/1500.0 loss: 0.14559602577847702 
Epoch:  1
1396/1500.0 loss: 0.1455064501690711 
Epoch:  1
1397/1500.0 loss: 0.14540993258255788 
Epoch:  1
1398/1500.0 loss: 0.1453205482998305 
Epoch:  1
1399/1500.0 loss

In [111]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss() #binomial cross entropy loss
        loss = loss_func(logits, labels) 
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99       249
           1       1.00      0.99      0.99       251

    accuracy                           0.99       500
   macro avg       0.99      0.99      0.99       500
weighted avg       0.99      0.99      0.99       500



PRECISION = TRUE POS / TRUE POS + FALSE POS
RECALL = TRUE POS / TRUE POS + FALSE NEG
F1 = (2 * RECALL * PRECISION) / ( RECALL + PRECISION)